# Agent Executor From Scratch

In this notebook we will go over how to build a basic agent executor from
scratch.

![diagram](./img/agent-executor-diagram.png)

## Setup

First we need to install the packages required

```bash
yarn add @langchain/community @langchain/anthropic @langchain/langgraph @langchain/core
```

Next, we need to set API keys for Anthropic (the LLM we will use) and Tavily (the
search tool we will use)

```bash
export ANTHROPIC_API_KEY=
export TAVILY_API_KEY=
```

Optionally, we can set API key for
[LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.

```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=
```

## Define the graph schema

We first need to define the graph state. The state for a traditional LangGraph agent has a single attribute, `messages` which holds the original input, as well as the conversation history.

In [21]:
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

## Define tools

Next, we will define the tools we'll use in this LangGraph graph. For this example, we'll use the pre-build Tavily Search tool, however you can use any pre-built, or custom tool you'd like. See [this guide](https://js.langchain.com/docs/how_to/custom_tools/) on how to create custom LangChain tools.

We'll want to pass our tools to the `ToolNode`, which is the way LangGraph executes tools.

In [ ]:
import { TavilySearch } from "@langchain/tavily";
import { ToolNode } from "@langchain/langgraph/prebuilt";

const tools = [new TavilySearch({ maxResults: 1 })];

const toolNode = new ToolNode<typeof AgentState.State>(tools);

## Define the nodes

We now need to define a few different nodes in our graph. In `langgraph`, a node
can be either a function or a
[runnable](https://js.langchain.com/docs/expression_language/). There are two
main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node
   will then execute that action.

We will also need to define some edges. Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of
several paths may be taken. The path that is taken is not known until that node
is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either: a. If the
   agent said to take an action, then the function to invoke tools should be
   called b. If the agent said that it was finished, then it should finish
2. Normal Edge: after the tools are invoked, it should always go back to the
   agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional
edge to take.

In [23]:
import type { RunnableConfig } from "@langchain/core/runnables";
import { ChatAnthropic } from "@langchain/anthropic";
import { END } from "@langchain/langgraph";

// Define the LLM to be used in the agent
const llm = new ChatAnthropic({
  model: "claude-3-5-sonnet-20240620",
  temperature: 0,
}).bindTools(tools); // Ensure you bind the same tools passed to the ToolExecutor to the LLM, so these tools can be used in the agent

// Define logic that will be used to determine which conditional edge to go down
const shouldContinue = (data: typeof AgentState.State): "executeTools" | typeof END => {
  const { messages } = data;
  const lastMsg = messages[messages.length - 1];
  // If the agent called a tool, we should continue. If not, we can end.
  if (!("tool_calls" in lastMsg) || !Array.isArray(lastMsg.tool_calls) || !lastMsg?.tool_calls?.length) {
    return END;
  }
  // By returning the name of the next node we want to go to
  // LangGraph will automatically route to that node
  return "executeTools";
};

const callModel = async (data: typeof AgentState.State, config?: RunnableConfig): Promise<Partial<typeof AgentState.State>> => {
  const { messages } = data;
  const result = await llm.invoke(messages, config);
  return {
    messages: [result],
  };
};


## Define the graph

We can now put it all together and define the graph!

In [24]:
import { START, StateGraph } from "@langchain/langgraph";

// Define a new graph
const workflow = new StateGraph(AgentState)
  // Define the two nodes we will cycle between
  .addNode("callModel", callModel)
  .addNode("executeTools", toolNode)
  // Set the entrypoint as `callModel`
  // This means that this node is the first one called
  .addEdge(START, "callModel")
  // We now add a conditional edge
  .addConditionalEdges(
    // First, we define the start node. We use `callModel`.
    // This means these are the edges taken after the `agent` node is called.
    "callModel",
    // Next, we pass in the function that will determine which node is called next.
    shouldContinue,
  )
  // We now add a normal edge from `tools` to `agent`.
  // This means that after `tools` is called, `agent` node is called next.
  .addEdge("executeTools", "callModel");

const app = workflow.compile();

In [29]:
import { HumanMessage, BaseMessage } from "@langchain/core/messages";

let finalResult: BaseMessage | undefined;

const prettyLogOutput = (output: Record<string, any>) => {
  const keys = Object.keys(output);
  const firstItem = output[keys[0]];
  if ("messages" in firstItem) {
    console.log(`(node) ${keys[0]}:`, firstItem.messages[0]);
    console.log("----\n");
  }
}

const inputs = { messages: [new HumanMessage("Search the web for the weather in sf")] };
for await (const s of await app.stream(inputs)) {
  prettyLogOutput(s);
  if ("callModel" in s && s.callModel.messages?.length) {
    finalResult = s.callModel.messages[0];
  }
}

console.log("Final Result: ", finalResult.content)

(node) callModel:  AIMessage {
  "id": "msg_01SwpMu2zu8W4NZeLEg5DHKj",
  "content": [
    {
      "type": "text",
      "text": "Certainly! I'll use the Tavily search engine to find current weather information for San Francisco (SF). Let me search for that information for you."
    },
    {
      "type": "tool_use",
      "id": "toolu_013Asn4HooNPMtYYapQPcewB",
      "name": "tavily_search_results_json",
      "input": {
        "input": "current weather in San Francisco"
      }
    }
  ],
  "additional_kwargs": {
    "id": "msg_01SwpMu2zu8W4NZeLEg5DHKj",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "tool_use",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 416,
      "output_tokens": 94
    }
  },
  "response_metadata": {
    "id": "msg_01SwpMu2zu8W4NZeLEg5DHKj",
    "model": "claude-3-5-sonnet-20240620",
    "stop_reason": "tool_use",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 416,